In [ ]:
!gcloud dataproc clusters list --region us-central1

In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [2]:
import pyspark
import builtins
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from timeit import timeit
import random
import numpy as np
from collections import defaultdict

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

import ast
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from contextlib import closing
porter = PorterStemmer()

nltk.download('punkt')
nltk.download('stopwords')
stopwords = set(stopwords.words('english')) #stopwords list
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

stop = stopwords.union(corpus_stopwords)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp1.py

inverted_index_gcp1.py


In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  5 18:18 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp1.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [7]:
from inverted_index_gcp1 import InvertedIndex, MultiFileReader

In [8]:
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){,24}""", re.UNICODE)

def tokenize(text):
    token_words = [token.group() for token in RE_WORD.finditer(text)]
    stem_sentence=[]
    for word in token_words:
      if word.lower() not in stop and (word.isalpha() or word.isdigit()):
        stem_sentence.append(porter.stem(word.lower()))
    return stem_sentence

def simple_tokenize(text):
    token_words = [token.group() for token in RE_WORD.finditer(text)]
    stem_sentence=[]
    for word in token_words:
      if word not in stop and (word.isalpha() or word.isdigit()):
        stem_sentence.append(porter.stem(word.lower()))
    return stem_sentence

In [9]:
def word_count(id, tokens):
  counts = Counter()
  postinglist = []
  for word in tokens:
    counts[word] +=1
  for word, count in counts.items():
    postinglist.append((word,(id,count)))
  return postinglist

def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl, key = lambda x: x[1], reverse = True)

def reduce_word_counts1(unsorted_pl):
  ps = sorted(unsorted_pl, key = lambda x: len(x[1]), reverse = True)
  ps = [(x[0], x[1][:400]) for x in ps]
  return ps

def calculate_df(postings):
  ps = postings.map(lambda x: (x[0],len(x[1])))
  return ps

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings, inverted):
  ps = postings.map(lambda x: (token2bucket_id(x[0]), [(x[0], x[1])] ))
  ps = ps.reduceByKey(lambda accum, n: accum + n ).mapValues(reduce_word_counts1)
  ps = ps.map(lambda x: inverted.write_a_posting_list(x , bucket_name))
  return ps

TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer

def read_posting_list(inverted, w, bucket_name):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE, bucket_name)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list


def rare_words(tokens):
    t = []
    for token in tokens:
        if postings_not_filtered[token] != 1:
            t.append(token)
    return t

In [10]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
doc_text_pairs = parquetFile.select("id", "text").rdd
tokens_pages = doc_text_pairs.map(lambda x: (x[0], tokenize(x[1])))

In [11]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'text_315923151'
client = storage.Client()
blobs = client.list_blobs(bucket_name)

for b in blobs:
    print(b.name)

NUM_BUCKETS = 248

graphframes.sh
startup_script_gcp.sh


In [ ]:
text_InvertedIndex = InvertedIndex()

# for each word we save how many times each word appear in each doc
word_counts = tokens_pages.flatMap(lambda x: word_count(x[0], x[1]))

# tuple of word and posting list(each posting list sort by doc id)
postings = word_counts.groupByKey()

# we will filter out rare words, words that appear in 50 or fewer documents.
postings_filtered = postings.filter(lambda x: len(x[1]) > 50).mapValues(reduce_word_counts)

# len of each document
text_InvertedIndex.len_docs = Counter(tokens_pages.map(lambda x: (x[0], len(x[1]))).collectAsMap())

# number of documents
text_InvertedIndex.number_of_docs = len(text_InvertedIndex.len_docs)

# doc frequency for each term
text_InvertedIndex.df = Counter(calculate_df(postings_filtered).collectAsMap())

# posting list of td-idf for each word(top 400 sort by tf)
tf_idf_filtered = postings_filtered.map(lambda x: (x[0],[(y[0],(y[1]/text_InvertedIndex.len_docs[y[0]]) * np.log2(text_InvertedIndex.number_of_docs/text_InvertedIndex.df[x[0]])) for y in x[1]]))

# mehane for each doc_id
text_InvertedIndex.sum_idf = tf_idf_filtered.flatMap(lambda x: [(y[0],y[1]**2)  for y in x[1]]).reduceByKey(lambda x, y: x+y).collectAsMap()

# posting locs
posting_locs_list = partition_postings_and_write(postings_filtered, text_InvertedIndex).collect()

22/01/08 17:08:11 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 15.0 in stage 10.0 (TID 813) (cluster-d35d-w-1.us-central1-a.c.top-monitor-337308.internal executor 7): java.net.SocketException: Connection reset
	at java.net.SocketInputStream.read(SocketInputStream.java:210)
	at java.net.SocketInputStream.read(SocketInputStream.java:141)
	at java.io.BufferedInputStream.fill(BufferedInputStream.java:246)
	at java.io.BufferedInputStream.read(BufferedInputStream.java:265)
	at java.io.DataInputStream.readInt(DataInputStream.java:387)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:642)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:635)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:470)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scal

In [13]:
# collect all posting lists locations into one super-set
super_posting_locs_text = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_text[k].extend(v)
text_InvertedIndex.posting_locs = super_posting_locs_text

In [14]:
# write the global stats out
text_InvertedIndex.write_index('.', 'text_InvertedIndex')
# upload to gs
index_src = "text_InvertedIndex.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst
!gsutil ls -lh $index_dst

Copying file://text_InvertedIndex.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1 files][238.4 MiB/238.4 MiB]                                                
Operation completed over 1 objects/238.4 MiB.                                    
238.42 MiB  2022-01-08T17:12:06Z  gs://text_315923151/postings_gcp/tex